# * VINSIGHT : CEO Report

In [3]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [4]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100GEO' --Total Revenue (Geo)
        , 'B1R000100' --Prepaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB4S000500' --TVS Active Subs
         
        , 'B1S000101CS' --Prepaid Gross Adds
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB4S001400CS' --TVS Now Gross Adds
         
        , 'B1S000200' --Prepaid Churn Subs
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-10-21, 12:28:42

DataFrame: 15157 rows, 16 columns


In [5]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [6]:
# chk_src_df.tail()

## Period Parameter

In [7]:
''' Parameter '''

# v_tm_key_qtr = curr_qtr
v_tm_key_qtr = 20241

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

# v_tm_key_wk = curr_wk
v_tm_key_wk = last_3_wk
# v_tm_key_wk = 2025020

print(f'v_tm_key_qtr: {v_tm_key_qtr}')
print(f'v_tm_key_mth: {v_tm_key_mth}')
print(f'v_tm_key_wk: {v_tm_key_wk}')

v_tm_key_qtr: 20241
v_tm_key_mth: 202510
v_tm_key_wk: 2025040


## Revenue(Geo) by Period

### Yearly

In [8]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df[col] = rev_geo_yearly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df

,TM_KEY_YR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2024,2025-09-22 06:15:44,"142,125,302,342","48,319,930,436","70,959,878,334","18,827,249,425","4,018,244,147"
1,2025,2025-10-21 09:41:43,"113,938,206,448","39,919,779,553","55,726,681,141","15,416,335,102","2,643,431,882"


### Quarterly

In [9]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df[col] = rev_geo_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df

,TM_KEY_QTR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,20241,2025-09-22 06:15:44,"35,361,933,475","11,842,176,786","17,936,624,018","4,569,920,004","1,013,212,668"
1,20242,2025-09-22 06:15:44,"35,284,532,532","11,858,030,293","17,792,137,434","4,665,781,597","968,583,208"
2,20243,2025-09-22 06:15:44,"35,706,816,037","12,164,985,004","17,628,033,120","4,803,785,483","1,110,012,430"
3,20244,2025-09-22 06:15:44,"35,772,020,298","12,454,738,354","17,603,083,763","4,787,762,341","926,435,841"
4,20251,2025-10-21 09:41:43,"35,653,069,078","12,378,982,586","17,580,761,024","4,810,187,607","883,137,861"
5,20252,2025-10-21 09:41:43,"35,543,940,268","12,360,832,103","17,489,026,867","4,833,931,536","860,149,762"
6,20253,2025-10-21 09:41:43,"35,591,444,331","12,529,561,866","17,451,089,386","4,823,602,770","787,190,308"
7,20254,2025-10-21 09:41:43,"7,149,752,772","2,650,402,997","3,205,803,864","948,613,189","112,953,952"


### Monthly

In [10]:
''' Revenue(Geo) Monthly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df[col] = rev_geo_monthly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df

,TM_KEY_MTH,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202510,2025-10-21 09:41:43,"7,149,752,772","2,650,402,997","3,205,803,864","948,613,189","112,953,952"


### Weekly

In [11]:
''' Revenue(Geo) Weekly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]
# rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]
rev_geo_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = rev_geo_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df[col] = rev_geo_weekly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2025040,20251005,2025-10-21 09:41:43,"1,861,640,111","775,280,230","829,252,944","222,097,791","35,009,146"
1,2025041,20251012,2025-10-21 09:41:43,"2,678,851,540","939,839,028","1,319,861,765","382,700,985","36,449,761"
2,2025042,20251019,2025-10-21 09:41:43,"2,609,261,121","935,283,739","1,056,689,155","343,814,413","41,495,044"
3,2025043,20251020,2025-10-21 09:41:43,0,0,0,0,0


### Daily

In [12]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202510,20251020,2025-10-21 09:41:43,0,0,0,0,0
1,202510,20251019,2025-10-21 09:41:43,"214,462,304","136,383,271",0,0,0
2,202510,20251018,2025-10-21 09:41:43,"212,037,252","134,723,237",0,0,0
3,202510,20251017,2025-10-21 09:41:43,"215,776,292","139,190,571",0,0,0
4,202510,20251016,2025-10-21 09:41:43,"769,322,958","142,283,529","435,723,795","170,537,047","20,778,587"
5,202510,20251015,2025-10-21 09:41:43,"210,070,313","134,984,077","75,086,236",0,0
6,202510,20251014,2025-10-21 09:41:43,"200,388,375","125,732,027","74,656,349",0,0
7,202510,20251013,2025-10-21 09:41:43,"787,203,626","121,987,027","471,222,776","173,277,366","20,716,458"
8,202510,20251012,2025-10-21 09:41:43,"201,929,684","123,023,267","78,906,416",0,0
9,202510,20251011,2025-10-21 09:41:43,"210,610,250","129,568,488","81,041,763",0,0


## Subscriber by Period

### Yearly

In [13]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['FTTX_RPT'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df[col] = sub_yearly_df[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2024,2025-09-22 06:15:44,"28,100,141","14,158,698","3,154,889","1,180,410"
1,2025,2025-10-21 09:41:43,"25,640,509","13,925,006","3,227,333","1,065,355"


### Quarterly

In [14]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['FTTX_RPT'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df[col] = sub_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,20241,2025-09-22 06:15:44,0,0,"3,090,287","1,258,065"
1,20242,2025-09-22 06:15:44,"27,800,517","14,226,167","3,107,084","1,236,606"
2,20243,2025-09-22 06:15:44,"27,505,003","14,186,615","3,127,304","1,205,116"
3,20244,2025-09-22 06:15:44,"28,100,141","14,158,698","3,154,889","1,180,410"
4,20251,2025-10-21 09:41:43,"27,140,083","14,084,812","3,187,717","1,153,137"
5,20252,2025-10-21 09:41:43,"26,272,750","14,036,540","3,207,941","1,112,829"
6,20253,2025-10-21 09:41:43,"25,525,284","13,952,680","3,225,844","1,071,847"
7,20254,2025-10-21 09:41:43,"25,640,509","13,925,006","3,227,333","1,065,355"


### Monthly

In [15]:
''' Subscriber Monthly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['FTTX_RPT'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df[col] = sub_monthly_df[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202510,2025-10-21 09:41:43,"25,640,509","13,925,006","3,227,333","1,065,355"


### Weekly

In [16]:
''' Subscriber Weekly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]
# sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['FTTX_RPT'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]
sub_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = sub_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df[col] = sub_weekly_df[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2025040,20251005,2025-10-21 09:41:43,"25,542,856","13,953,938","3,227,049","1,070,257"
1,2025041,20251012,2025-10-21 09:41:43,"25,553,333","13,937,719","3,226,523","1,067,968"
2,2025042,20251019,2025-10-21 09:41:43,"25,640,509","13,925,006","3,227,333","1,065,355"


### Daily

In [17]:
''' Subscriber Daily '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]
sub_daily_df = sub_daily_df.loc[sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['FTTX_RPT'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df[col] = sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202510,20251020,2025-10-21 09:41:43,0,0,0,0
1,202510,20251019,2025-10-21 09:41:43,"25,640,509",0,0,"1,065,355"
2,202510,20251018,2025-10-21 09:41:43,"25,602,716","13,925,006","3,227,333","1,065,931"
3,202510,20251017,2025-10-21 09:41:43,"25,602,814","13,938,946","3,230,334","1,066,108"
4,202510,20251016,2025-10-21 09:41:43,"25,552,321","13,934,768","3,228,520","1,066,312"
5,202510,20251015,2025-10-21 09:41:43,"25,574,019","13,930,940","3,226,714","1,066,890"
6,202510,20251014,2025-10-21 09:41:43,"25,579,208","13,938,569","3,229,625","1,067,340"
7,202510,20251013,2025-10-21 09:41:43,"25,553,239","13,941,452","3,228,010","1,067,340"
8,202510,20251012,2025-10-21 09:41:43,"25,553,333","13,937,719","3,226,523","1,067,968"
9,202510,20251011,2025-10-21 09:41:43,"25,556,477","13,944,226","3,229,154","1,068,047"


## Gross Adds by Period

### Yearly

In [18]:
''' Gross Adds Yearly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE_GA'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_GA_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_GA_CONN_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW_GA'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df[col] = ga_yearly_df[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df

,TM_KEY_YR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2024,2025-09-22 02:25:18,"30,607,404","2,642,885","558,527","7,682"
1,2025,2025-10-21 03:33:31,"18,354,887","1,705,730","474,351","16,891"


### Quarterly

In [19]:
''' Gross Adds Quarterly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

ga_quarterly_df['PRE_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_GA_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_GA_CONN_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df[col] = ga_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,20241,2025-09-22 02:25:18,"8,953,743","701,773","130,958",0
1,20242,2025-09-22 02:25:18,"7,514,316","698,111","141,474",0
2,20243,2025-09-22 02:25:18,"7,156,127","662,059","147,285","4,992"
3,20244,2025-09-22 02:25:18,"6,983,218","580,942","138,810","2,690"
4,20251,2025-10-21 03:33:31,"6,673,875","569,340","145,692","3,337"
5,20252,2025-10-21 03:33:31,"5,586,276","550,281","152,388","6,328"
6,20253,2025-10-21 03:33:31,"5,127,389","494,607","146,023","5,920"
7,20254,2025-10-21 03:33:31,"967,347","91,502","30,248","1,306"


### Monthly

In [20]:
''' Gross Adds Monthly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_monthly_df['PRE_GA'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_GA_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_GA_CONN_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW_GA'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df[col] = ga_monthly_df[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202510,2025-10-21 03:33:31,"967,347","91,502","30,248","1,306"


### Weekly

In [21]:
''' Gross Adds Weekly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]
# ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

ga_weekly_df['PRE_GA'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_GA_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_GA_CONN_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW_GA'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]
ga_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = ga_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df[col] = ga_weekly_df[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2025040,20251005,2025-10-21 03:33:31,"280,851","24,596","8,069",302
1,2025041,20251012,2025-10-21 03:33:31,"345,762","36,216","11,194",501
2,2025042,20251019,2025-10-21 03:33:31,"340,734","30,690","10,985",503
3,2025043,20251020,2025-10-21 03:33:31,0,0,0,0


### Daily

In [22]:
''' Gross Adds Daily '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE_GA'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_GA_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_GA_CONN_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW_GA'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202510,20251020,2025-10-21 03:33:31,0,0,0,0
1,202510,20251019,2025-10-21 03:33:31,0,0,"1,631",0
2,202510,20251018,2025-10-21 03:33:31,"61,074","4,895","1,698",98
3,202510,20251017,2025-10-21 03:33:31,"66,370","4,982","1,548",90
4,202510,20251016,2025-10-21 03:33:31,"57,842","4,996","1,545",80
5,202510,20251015,2025-10-21 03:33:31,"54,050","6,219","1,522",79
6,202510,20251014,2025-10-21 03:33:31,"52,381","4,572","1,532",76
7,202510,20251013,2025-10-21 03:33:31,"49,017","5,026","1,509",80
8,202510,20251012,2025-10-21 03:33:31,"45,973","4,670","1,458",98
9,202510,20251011,2025-10-21 03:33:31,"48,600","4,873","1,646",73


## Churn Subs by Period

### Yearly

In [23]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_yearly_df = churn_yearly_df.loc[churn_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_yearly_df['PRE_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='B1S000200', churn_yearly_df['P'], 0)
churn_yearly_df['POST_CHURN_B2C'] = np.where(churn_yearly_df['METRIC_CD']=='B2S010200', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13100', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_VOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13102', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13103', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN(CF)'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13104', churn_yearly_df['P'], 0)
churn_yearly_df['TVS_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER14100', churn_yearly_df['P'], 0)

churn_yearly_df = churn_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_yearly_df = churn_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_yearly_df = churn_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_yearly_df[col] = churn_yearly_df[col].apply(lambda x: format(x, ',.0f'))
churn_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2024,2025-09-22 06:15:44,0,"2,857,216","456,668","170,154","281,222","282,188","194,250"
1,2025,2025-10-21 09:41:43,0,"1,838,376","340,751","133,878","236,430","284,052","181,730"


### Quarterly

In [24]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['METRIC_CD'].isin(v_metric_list)]
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

churn_quarterly_df['PRE_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='B1S000200', churn_quarterly_df['P'], 0)
churn_quarterly_df['POST_CHURN_B2C'] = np.where(churn_quarterly_df['METRIC_CD']=='B2S010200', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13100', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_VOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13102', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13103', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN(CF)'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13104', churn_quarterly_df['P'], 0)
churn_quarterly_df['TVS_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER14100', churn_quarterly_df['P'], 0)

churn_quarterly_df = churn_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_quarterly_df = churn_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_quarterly_df = churn_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_quarterly_df[col] = churn_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
churn_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,20241,2025-09-22 06:15:44,0,"871,395","118,742","43,223","72,945","73,317","48,913"
1,20242,2025-09-22 06:15:44,0,"718,051","114,261","43,677","70,807","70,972","40,970"
2,20243,2025-09-22 06:15:44,0,"664,232","118,872","44,269","67,909","68,985","48,795"
3,20244,2025-09-22 06:15:44,0,"603,538","104,793","38,985","69,561","68,914","55,572"
4,20251,2025-10-21 09:41:43,0,"553,494","104,815","38,083","68,600","68,746","47,799"
5,20252,2025-10-21 09:41:43,0,"586,573","92,588","43,483","70,494","105,284","66,735"
6,20253,2025-10-21 09:41:43,0,"590,961","121,642","44,568","80,563","82,283","67,196"
7,20254,2025-10-21 09:41:43,0,"107,348","21,706","7,744","16,773","27,739",0


### Monthly

In [25]:
''' Churn Subs Monthly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['METRIC_CD'].isin(v_metric_list)]
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_monthly_df['PRE_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='B1S000200', churn_monthly_df['P'], 0)
churn_monthly_df['POST_CHURN_B2C'] = np.where(churn_monthly_df['METRIC_CD']=='B2S010200', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13100', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_VOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13102', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13103', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN(CF)'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13104', churn_monthly_df['P'], 0)
churn_monthly_df['TVS_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER14100', churn_monthly_df['P'], 0)

churn_monthly_df = churn_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_monthly_df = churn_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_monthly_df = churn_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_monthly_df[col] = churn_monthly_df[col].apply(lambda x: format(x, ',.0f'))
churn_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202510,2025-10-21 09:41:43,0,"107,348","21,706","7,744","16,773","27,739",0


### Weekly

In [26]:
''' Churn Subs Weekly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['METRIC_CD'].isin(v_metric_list)]
# churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

churn_weekly_df['PRE_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='B1S000200', churn_weekly_df['P'], 0)
churn_weekly_df['POST_CHURN_B2C'] = np.where(churn_weekly_df['METRIC_CD']=='B2S010200', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13100', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_VOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13102', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13103', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN(CF)'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13104', churn_weekly_df['P'], 0)
churn_weekly_df['TVS_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER14100', churn_weekly_df['P'], 0)

churn_weekly_df = churn_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_weekly_df = churn_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_weekly_df = churn_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]
churn_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = churn_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_weekly_df[col] = churn_weekly_df[col].apply(lambda x: format(x, ',.0f'))
churn_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2025040,20251005,2025-10-21 09:41:43,0,"23,425","6,581","2,424","6,635","27,739",0
1,2025041,20251012,2025-10-21 09:41:43,0,"40,561","11,483","3,178","4,310",0,0
2,2025042,20251019,2025-10-21 09:41:43,0,"43,362","3,642","2,142","5,828",0,0
3,2025043,20251020,2025-10-21 09:41:43,0,0,0,0,0,0,0


### Daily

In [27]:
''' Churn Subs Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_daily_df = churn_daily_df.loc[churn_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_daily_df = churn_daily_df.loc[churn_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_daily_df['PRE_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='B1S000200', churn_daily_df['P'], 0)
churn_daily_df['POST_CHURN_B2C'] = np.where(churn_daily_df['METRIC_CD']=='B2S010200', churn_daily_df['P'], 0)
churn_daily_df['FTTX_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13100', churn_daily_df['P'], 0)
churn_daily_df['FTTX_VOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13102', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13103', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN(CF)'] = np.where(churn_daily_df['METRIC_CD']=='TSER13104', churn_daily_df['P'], 0)
churn_daily_df['TVS_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER14100', churn_daily_df['P'], 0)

churn_daily_df = churn_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_daily_df = churn_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_daily_df = churn_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_daily_df[col] = churn_daily_df[col].apply(lambda x: format(x, ',.0f'))
churn_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202510,20251020,2025-10-21 09:41:43,0,0,0,0,0,0,0
1,202510,20251019,2025-10-21 09:41:43,0,0,0,0,0,0,0
2,202510,20251018,2025-10-21 09:41:43,0,"13,208",0,0,0,0,0
3,202510,20251017,2025-10-21 09:41:43,0,"5,022",-304,422,"1,947",0,0
4,202510,20251016,2025-10-21 09:41:43,0,"3,916",-300,401,0,0,0
5,202510,20251015,2025-10-21 09:41:43,0,"6,321","4,399",454,0,0,0
6,202510,20251014,2025-10-21 09:41:43,0,"10,721",-115,401,"3,881",0,0
7,202510,20251013,2025-10-21 09:41:43,0,"4,174",-38,464,0,0,0
8,202510,20251012,2025-10-21 09:41:43,0,"4,774","4,154",468,1,0,0
9,202510,20251011,2025-10-21 09:41:43,0,"2,852",31,418,0,0,0
